In [1]:
import cv2
import time
import numpy as np
import os
import json

class GenTrainingJson(object):

    def generateJson(self, dir_label):  #[{dir: 'warriorPoseI/', label: 'WarriorPoseI'}, {...}, ... ]
        data = {"data": []}

        for obj in dir_label:
            poseDic = {}
            for filename in os.scandir(obj['dir']):
                if (filename.path.endswith(".jpg") or filename.path.endswith(".png")) and filename.is_file():
                    pointsList = self.predictImageKeypoints(filename.path)
                    poseDic['xs'] = {i:pointsList[i] for i in range(0, len(pointsList) ) }
                    poseDic['ys'] = {"0":obj['label']}
                    data['data'].append(poseDic)
        return data
                    
    def arrayToOneDim(self, List):
        newList = []
        for item in List: 
            if item == None:
                newList.append((0,0))
            else:
                newList.append(item)

        newList = [element for tupl in newList for element in tupl]
        return newList
    
    def swapIndices(self, kps):
        seq = [0,15,14,17,16,5,2,6,3,7,4,11,8,12,9,13,10] #reorder the index for posenet
        arr = []
        for i in seq:
            arr.append(kps[i])
        return arr
        
    def predictImageKeypoints(self, img_path):
        protoFile = "pose/coco/pose_deploy_linevec.prototxt"
        weightsFile = "pose/coco/pose_iter_440000.caffemodel"
        nPoints = 18
        POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]
            
        frame = cv2.imread(img_path)
        frameCopy = np.copy(frame)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        threshold = 0.1

        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

        t = time.time()
        # input image dimensions for the network
        inWidth = 640
        inHeight = 480
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)

        output = net.forward()
        print("time taken by network : {:.3f}".format(time.time() - t))

        H = output.shape[2]
        W = output.shape[3]

        # Empty list to store the detected keypoints
        points = []

        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

                # Add the point to the list if the probability is greater than the threshold
                points.append((float(x), float(y)))
            else :
                points.append((0,0))

#         points = self.swapIndices(points) # turn on for posenet
        points.pop() #turn on for open pose

        points = [element for tupl in points for element in tupl]
        return points



In [2]:
directoryList = [
    {
        'dir': 'warriorPoseI/',
        'label': 'WarriorPoseI'
    },
    {
        'dir': 'warriorPoseII/',
        'label': 'WarriorPoseII'
    },
    {
        'dir': 'warriorPoseIII/',
        'label': 'WarriorPoseIII'
    },    
    {
        'dir': 'TreePose/',
        'label': 'TreePose'
    },
]

generator = GenTrainingJson()
data = generator.generateJson(directoryList)

with open('PoseNettrain.json', 'w') as fp:
    json.dump(data, fp)

time taken by network : 4.681
17
[0, 0, 503.3625, 374.8333333333333, 519.6, 374.8333333333333, 438.4125, 259.5, 503.3625, 187.41666666666666, 503.3625, 374.8333333333333, 438.4125, 245.08333333333334, 454.65, 144.16666666666666, 470.8875, 547.8333333333334, 373.4625, 547.8333333333334, 324.75, 692.0, 519.6, 562.25, 633.2625, 648.75, 811.875, 677.5833333333334, 0, 0, 470.8875, 288.3333333333333, 0, 0]
time taken by network : 4.713
17
[0, 0, 481.95, 640.0, 495.3375, 613.3333333333334, 508.725, 426.6666666666667, 428.4, 266.6666666666667, 468.5625, 666.6666666666666, 415.0125, 453.3333333333333, 334.6875, 293.3333333333333, 575.6625, 853.3333333333334, 307.9125, 1146.6666666666667, 200.8125, 1306.6666666666667, 589.05, 880.0, 696.15, 960.0, 736.3125, 1120.0, 0, 0, 0, 0, 455.175, 533.3333333333334]


KeyboardInterrupt: 

In [17]:
with open('PoseNettrain.json', 'w') as fp:
    json.dump(data, fp)